In [2]:
import HelperFuncs as hfuncs
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils.data_utils import Sequence
import h5py
import os

Define a class that allows us to sequentially load entire scans. 

In [2]:
BATCH_SIZE = 20
FINAL_WIDTH = 400
FINAL_HEIGHT = 600
CHANNELS = 1
ZONES = 17

class ScanSequencer(Sequence):
    idx_dict={}
    
    def __init__(self,num_batches,bucket_name,mode="train_scan"):
        self.num_batches = num_batches
        self.bucket_name = bucket_name
        self.mode = mode
        self.key_id, self.secret_key = hfuncs.GetAWSCredentials()
        self.mode = mode
    def __len__(self):
        return self.num_batches
    def __getitem__(self,idx):
        #Get Client
        client = hfuncs.GetAWSClient(self.key_id,self.secret_key)
        bucket = client.Bucket(self.bucket_name)
        
        #Download batch at index
        path = "temp/{}/batch_{}.hdf5".format(self.mode,idx)
        key = "{}/batch_{}.hdf5".format(self.mode,idx)
        bucket.download_file(Key=key,Filename=path)
        
        f = h5py.File(path,"r")
        try:
            return f['/image'].value, f['/labels'].value
        finally:
            f.close()
            os.remove(path) 
         



In [9]:
import h5py
from keras.models import load_model

#Load trained model
BASE_MODEL = load_model("Final_CNNRes.h5")

In [7]:
from inception_resnet_v1 import InceptionResNetV1
from keras.layers import Input,Flatten,Dense,Concatenate,Dropout
from keras.models import Model
import keras
from datetime import datetime
from keras.callbacks import TensorBoard,EarlyStopping
from keras.optimizers import SGD,Adam
from keras import metrics
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import LSTM

def trainCNNResRNNAdam(alpha,lr,beta1,beta2,pooling,dropout_rate,description="RNN"):
    #Number of angles per scan 
    ANGLES = 32

    #Hidden dimensions
    LSTM_OUTPUT_DIM = 1000

    #Build new sequential model,Removing dense layers from model
    input_scan = Input(shape=(ANGLES,FINAL_WIDTH,FINAL_HEIGHT,CHANNELS))
    
    #From our base model, we'll remove 18 extraneous layers (one concatenation and 17 dense) since these
    #were only used for making predictions for each of the zones using the extracted features.  Now, we'll be making 
    #predictions only after we've seen all of the supplied angle slices. 
    sequenced_model = TimeDistributed(BASE_MODEL.layers[-19])(input_scan)

    #One lstm layer for now
    lstm = LSTM(LSTM_OUTPUT_DIM)(sequenced_model)

    #Finally, 17 dense layers connected to the output
    output_nodes = []
    for i in range(ZONES):
        output_nodes.append(Dense(1,activation='sigmoid')(lstm))

    out = keras.layers.concatenate(output_nodes)

    #complete model
    recurrent_model = Model(input_scan, out)
   
    #optimizer
    adam = Adam(lr,beta_1=beta1,beta_2=beta2)
    multi_label_model.compile(optimizer=adam,
                              metrics=[metrics.binary_accuracy],
                             loss= 'binary_crossentropy')

    #Tensorboard
    x = datetime.today()
    stamp = "{}-{}-{}_{}:{}:{}_lr-{}_beta1-{}_beta2-{}_alpha-{}_pooling-{}_dropout-{}".format(x.year,x.month,
                                                         x.day,x.hour,x.minute,
                                                         x.second,lr,beta1,beta2,alpha,pooling,dropout_rate)
    tensorboard = TensorBoard(log_dir="logs/{}".format(stamp),histogram_freq=0,batch_size=BATCH_SIZE,
                              write_grads=False,write_images=True)

    #Generators and fit

    #Bucket with clean data
    UPLOAD_BUCKET = 'cleandhsdata' #bucket where clean data was stored
    key_id, secret_key = hfuncs.GetAWSCredentials()
    client = hfuncs.GetAWSClient(key_id,secret_key)
    bucket = client.Bucket(UPLOAD_BUCKET)

    #Initialize train sequencer
    mode ="train_scan"
    num_batches = sum([1 if "{}/".format(mode) in k.key else 0 for k in bucket.objects.all()])-1 #train,test,val root directories have their own keys
    train_seq = Sequencer(num_batches,UPLOAD_BUCKET,mode=mode)

    #Initialize validation sequencer
    mode = "val_scan"
    num_batches = sum([1 if "{}/".format(mode) in k.key else 0 for k in bucket.objects.all()])-1 #train,test,val root directories have their own keys
    val_seq = Sequencer(num_batches,UPLOAD_BUCKET,mode=mode)

    #Early stopping callback
    estop = EarlyStopping(monitor='loss',min_delta=0.005,patience=3)
    
    hist = multi_label_model.fit_generator(train_seq,
                                           steps_per_epoch=20,
                                           epochs=5,
                                           validation_data = val_seq,
                                           validation_steps = 50,
                                           callbacks=[tensorboard,estop],
                                          use_multiprocessing =True,workers=4)
    return hist,multi_label_model

Load moodel that we trained on individual slices. 

Now, we can remove the top layers of the model since these were used for making predictions at the slice level.  We know we can get rid of 18 superfluous layers, one concatenation, and 17 dense layers used to make predictions for each of the 17 regions. 

In [22]:
print(base_model.layers[-20].get_config())
print(base_model.get_layer(name='global_max_pooling2d_3').get_config())

{'data_format': 'channels_last', 'name': 'global_max_pooling2d_3', 'trainable': True}
{'data_format': 'channels_last', 'name': 'global_max_pooling2d_3', 'trainable': True}


In [23]:
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import LSTM
from keras.layers import Input,Flatten,Dense,Concatenate,Dropout

#Number of angles per scan 
ANGLES = 32

#Hidden dimensions
LSTM_OUTPUT_DIM = 1000

#Build new sequential model,Removing dense layers from model
input_scan = Input(shape=(ANGLES,FINAL_WIDTH,FINAL_HEIGHT,CHANNELS))
sequenced_model = TimeDistributed(BASE_MODEL.get_layer(name='global_max_pooling2d_3'))(input_scan)

#One lstm layer for now
lstm = LSTM(LSTM_OUTPUT_DIM)(sequenced_model)

#Finally, 17 dense layers connected to the output
output_nodes = []
for i in range(ZONES):
    output_nodes.append(Dense(1,activation='sigmoid')(lstm))

out = keras.layers.concatenate(output_nodes)

#complete model
multi_label_model = Model(input_scan, out)



Let's try training the model